In [3]:
import pandas as pd

import modulo_producciones as prod

In [4]:
prod.ModuloProducciones()

In [26]:
INPUT_FILEPATH = "input/produccion_junio_2023.xlsx"
MES_A_ANALIZAR = "June"

In [42]:
servicios = []
archivo = pd.ExcelFile(INPUT_FILEPATH)
for hoja in archivo.sheet_names[3:]:
    df = pd.read_excel(archivo, sheet_name=hoja)
    df = df.loc[:, "Rut":"Hoja1.Nombre nuevo HIS"]
    df["Hoja1.Nombre nuevo HIS"] = df["Hoja1.Nombre nuevo HIS"].str.strip().str.lower()
    procedimientos_mes = df[df["Fecha Realización"].dt.month_name() == MES_A_ANALIZAR]
    procedimientos_por_servicio = procedimientos_mes.groupby("Hoja1.Nombre nuevo HIS")[
        "Rut"
    ].count().reset_index(name="cantidad_procedimientos")

    servicios += list(df["Hoja1.Nombre nuevo HIS"].unique())



In [43]:
set(servicios)

{'ambulatorio',
 'medico quirugico 3° norte y sur',
 'medico quirugico 4° norte',
 'uci 4° sur',
 'uci 5° norte',
 'uci 5° sur',
 'uti 3° norte',
 'uti 4° norte',
 'uti 4° sur',
 'uti 5° norte'}

In [9]:
CORRELACION_PROCEDIMIENTOS_SIGCOM = {
    "QMT": "PROCEDIMIENTO ONCOLOGÍA",
    "APNEA": "PROCEDIMIENTO DE NEUMOLOGÍA",
    "EF": "PROCEDIMIENTOS DE CARDIOLOGÍA",
    "EBUS": "PROCEDIMIENTOS EBUS",
    "TAVI": "PROCEDIMIENTOS TAVI",
    "ASIST VENTRICULAR": "PROCEDIMIENTOS ASISTENCIA VENTRICULAR",
    "ECMO": "PROCEDIMIENTOS ECMO",
    "HMD": "PROCEDIMIENTOS DE HEMODINAMIA",
    "BRONCOSCOPIA": "PROCEDIMIENTOS ENDOSCÓPICOS",
}

SERVICIO_INTERNO_SIGCOM = {
    "ambulatorio": "CONSULTAS",
    "medico quirugico 3° norte y sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACIÓN QUIRÚRGICA",
    "medico quirugico 4° norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "uci 4° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° norte": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uti 3° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° sur": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 5° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
}